In [1]:
from exact_concorde.exact_concorde import *
from heuristic_LKH.heuristic_LKH import LKH
import os
os.environ["QSOPT_DIR"] = os.path.abspath("../pyconcorde/data/")
import sys
sys.path.append('./')

[WARNING] CuPy not found. Falling back to NumPy.


In [2]:
file_parser = FileParser()
solution_plotter = SolutionPlot()

In [34]:
task = 'd1291'
fname = f'./instances/tsplib/{task}.tsp'
# fname = './instances/tsplib/a280.tsp'

instance_info = file_parser.parse_instance_from_file(fname)
coordinates = instance_info['COORDINATES']
distance_mat = np.array(instance_info['COST_MATRIX']) # empty
nodes = {i: (x, y) for i, (x, y) in enumerate(coordinates)}

tsp_instance = TravelingSalesmenProblem(node_coords_dict=nodes)

lkh_model = LKH(list(nodes.keys()), coordinates)
lkh_model.optimize()

100%|█████████▉| 1297/1298 [00:00<00:00, 688870.75it/s]


Problem Name: Pseudo_TSP_Instance
Problem Type: TSP
Number of Nodes: 1291
Rounded Euclidean Norm (CC_EUCLIDEAN)


In [3]:
# calculate route
def cal_route_dist_lkh(route, task):
    file_parser = FileParser()
    instance_info = file_parser.parse_instance_from_file(f"instances/tsplib/{task}.tsp")
    node_coords = {i: np.array([x[0], x[1]]) for i, x in enumerate(instance_info["COORDINATES"])}

    total_dist = 0
    for i in range(len(route) - 1):
        node1 = node_coords[route[i]]
        node2 = node_coords[route[i+1]]
        total_dist += np.floor(np.linalg.norm(node1 - node2) + 0.5)
    print("model travel distance:", total_dist)

In [8]:
task = 'd15112'
route = [int(x.strip()) -1 for x in open('experiments/LLM_TSP_exp/d15112_traces_max_nodes_1000_time_budget_2000_initial_LKH_llm_gpt-5-mini-2025-08-07_gpt-5-2025-08-07_solver_concorde_subproblem_2_parallel_workers.txt').readlines()[4:]]
route[-1] = route[0]
cal_route_dist_lkh(route, task)

100%|█████████▉| 15118/15119 [00:00<00:00, 155542.74it/s]

model travel distance: 1590493.0


In [37]:
cal_route_dist_lkh(lkh_model.route, task)

100%|█████████▉| 1297/1298 [00:00<00:00, 577679.97it/s]

model travel distance: 51112.0


In [38]:
# calculate optimality gap
def cal_opt_gap(model_route_dist, taskname):
    opt = int(open('./instances/opt/{}.opt.tour.txt'.format(taskname)).readlines()[-1].strip())
    return (model_route_dist - opt) / opt

In [39]:
cal_opt_gap(51112, task)


0.0061219267337257145

Optimality gap for LKH-3 on Task d1291: 0.61%